### 1导包

In [2]:
from io import StringIO
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd
import numpy as np
import os
import sys
import math
import model
import torch
import requests
import pickle
import numpy as np
from rdkit import Chem
from collections import defaultdict

### 2访问uniport

In [5]:
#go2uniprot是一个类，他的目的是用来访问uniprot网址的
class go2uniprot(object):
    def __init__(self,query_parameter="",ec_parameter="",length_parameter=["0","2A"],  \
                       keys_words_list="",gene=""):
        """初始化属性"""
        
        self.query_parameter = query_parameter.replace(" ","%20")
        self.ec_parameter = ec_parameter
        self.length_parameter = length_parameter
        self.keys_words_list = keys_words_list
        self.gene = gene
        
        self.basic_url = "https://rest.uniprot.org/uniprotkb/stream?compressed=false&format=fasta&query="
        self.informal_url =""
        
        if keys_words_list == "" and gene == "":
            #如果没有关键词 和基因名 则是这这样的访问形式
            self.informal_url = self.first_query_way()
            
            #如果没有关键词，但是有基因名，则这样访问
        elif keys_words_list == "" and gene != "":
            self.informal_url = self.third_query_way()
            
        else:
            self.informal_url = self.second_query_way()
            
            
    def first_query_way(self):
        #第一重方位方式 如果没有提供EC编号

        if self.ec_parameter != "":
            informal_url = self.basic_url + self.query_parameter        + "%20AND%20%28ec%3A" \
                                             + self.ec_parameter        + "%29%20AND%20%28length%3A%5B" \
                                             + self.length_parameter[0] + "%20TO%20%"   \
                                             + self.length_parameter[1] + "%5D%29"
        else:
            informal_url  = self.basic_url + self.query_parameter       +  "%20AND%20%28length%3A%5B" \
                                             + self.length_parameter[0] + "%20TO%20%"   \
                                             + self.length_parameter[1] + "%5D%29"
        print(informal_url)
        
        return informal_url
        
        
    def second_query_way(self):
        #这个暂时没有用
        pass
#         for i in range(len(self.eys_words_list)):
#             [keys_words for i in range(len(self.keys_words_list)) if self.keys_words[i][2] != ""]
            
            
        
#         print(self.informal_url)

    def third_query_way(self):
        if self.ec_parameter != "":
            informal_url = self.basic_url + self.query_parameter        + "%20AND%20%28ec%3A" \
                                             + self.ec_parameter        + "%29%20AND%20%28length%3A%5B" \
                                             + self.length_parameter[0] + "%20TO%20%"   \
                                             + self.length_parameter[1] + "%5D%29%20AND%20%28gene%3A" \
                                             + self.gene                + "%29"
        else:
            informal_url  = self.basic_url + self.query_parameter       +  "%20AND%20%28length%3A%5B" \
                                             + self.length_parameter[0] + "%20TO%20%"   \
                                             + self.length_parameter[1] + "%5D%29%20AND%20%28gene%3A" \
                                             + self.gene                + "%29"
        print(informal_url)
        
        return informal_url

        
    #这个函数是用来去访问uniprot
    def ping2uniprot(self):
        """连接uniprot"""
#         fasta_result_text = requests.get(self.informal_url).text
        fasta_result_text = requests.get(self.informal_url).text
        return fasta_result_text

    def fasta_text2SeqRecord(self):
        """这里是为了方便使用BioPython，所以转换数据格式"""
        Seq_files = StringIO(self.ping2uniprot())
        pSeq = SeqIO.parse(Seq_files, 'fasta') 
        

        
#         name_seq_dictionary = {}
        name_seq_list = []
        OS_seq_list = []
        if self.gene == "":
            for AAseq in pSeq: 
            #     print(AAseq.seq)
            #     print(AAseq.name.split("|")[1])
            #     break
                AAseq_SeqRecord =  SeqRecord(Seq(str(AAseq.seq)), \
                                             id=str(AAseq.name.split("|")[1])) 
                name_seq_list.append(AAseq_SeqRecord)


                a = AAseq.description
                a = a.split("OS=")[1].split(" OX")[0]
                name_AAseq_SeqRecord =  SeqRecord(seq=" ",name=a, \
                                             id=str(AAseq.name.split("|")[1])) 

                OS_seq_list.append(name_AAseq_SeqRecord)
        else:
            for AAseq in pSeq: 
                AAseq_SeqRecord =  SeqRecord(Seq(str(AAseq.seq)), \
                                             id=str(AAseq.name.split("|")[1]),description=self.gene) 
                name_seq_list.append(AAseq_SeqRecord)


                a = AAseq.description
                a = a.split("OS=")[1].split(" OX")[0]
                name_AAseq_SeqRecord =  SeqRecord(seq=" ",name=a, \
                                             id=str(AAseq.name.split("|")[1])) 

                OS_seq_list.append(name_AAseq_SeqRecord)
            
        
                
        return name_seq_list,OS_seq_list

### 3输入关键词

In [6]:
#关键词Purine nucleoside phosphorylase
aaa,aaa_OS=go2uniprot("Purine nucleoside phosphorylase","2.4.2.1",gene="PNP").fasta_text2SeqRecord()
bbb,bbb_OS=go2uniprot("Purine nucleoside phosphorylase","2.4.2.1",gene="LACC1").fasta_text2SeqRecord()
ccc,ccc_OS=go2uniprot("PUNPI ","2.4.2.1",gene="PNP").fasta_text2SeqRecord()
ddd,ddd_OS=go2uniprot("PUNPI ","2.4.2.1",gene="LACC1").fasta_text2SeqRecord()
eee,eee_OS=go2uniprot("purine deoxynucleoside phosphorylase","2.4.2.1",gene="PNP").fasta_text2SeqRecord()
fff,fff_OS=go2uniprot("purine deoxynucleoside phosphorylase","2.4.2.1",gene="LACC1").fasta_text2SeqRecord()
ggg,ggg_OS=go2uniprot("inosine phosphorylase","2.4.2.1",gene="PNP").fasta_text2SeqRecord()
hhh,hhh_OS=go2uniprot("inosine phosphorylase","2.4.2.1",gene="LACC1").fasta_text2SeqRecord()
iii,iii_OS=go2uniprot("inosine-guanosine phosphorylase","2.4.2.1",gene="PNP").fasta_text2SeqRecord()
jjj,jjj_OS=go2uniprot("inosine-guanosine phosphorylase","2.4.2.1",gene="LACC1").fasta_text2SeqRecord()
kkk,kkk_OS=go2uniprot("purine ribonucleoside phosphorylase","2.4.2.1",gene="PNP").fasta_text2SeqRecord()
lll,lll_OS=go2uniprot("purine ribonucleoside phosphorylase","2.4.2.1",gene="LACC1").fasta_text2SeqRecord()

https://rest.uniprot.org/uniprotkb/stream?compressed=false&format=fasta&query=Purine%20nucleoside%20phosphorylase%20AND%20%28ec%3A2.4.2.1%29%20AND%20%28length%3A%5B0%20TO%20%2A%5D%29%20AND%20%28gene%3APNP%29
https://rest.uniprot.org/uniprotkb/stream?compressed=false&format=fasta&query=Purine%20nucleoside%20phosphorylase%20AND%20%28ec%3A2.4.2.1%29%20AND%20%28length%3A%5B0%20TO%20%2A%5D%29%20AND%20%28gene%3ALACC1%29
https://rest.uniprot.org/uniprotkb/stream?compressed=false&format=fasta&query=PUNPI%20%20AND%20%28ec%3A2.4.2.1%29%20AND%20%28length%3A%5B0%20TO%20%2A%5D%29%20AND%20%28gene%3APNP%29
https://rest.uniprot.org/uniprotkb/stream?compressed=false&format=fasta&query=PUNPI%20%20AND%20%28ec%3A2.4.2.1%29%20AND%20%28length%3A%5B0%20TO%20%2A%5D%29%20AND%20%28gene%3ALACC1%29
https://rest.uniprot.org/uniprotkb/stream?compressed=false&format=fasta&query=purine%20deoxynucleoside%20phosphorylase%20AND%20%28ec%3A2.4.2.1%29%20AND%20%28length%3A%5B0%20TO%20%2A%5D%29%20AND%20%28gene%3APNP%29
https

In [7]:
print(len(aaa)) #序列数目
print(len(bbb))
print(len(ccc))
print(len(ddd))
print(len(eee))
print(len(fff))
print(len(ggg))
print(len(hhh))
print(len(iii))
print(len(jjj))
print(len(kkk))
print(len(lll))

733
482
0
0
0
0
710
482
710
39
585
0


In [8]:
all_fasta_result_list = aaa+bbb+ccc+ddd+eee+fff+ggg+hhh+iii+jjj+kkk+lll  #蛋白序列
all_fasta_result_list

[SeqRecord(seq=Seq('MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLTDKLTEAQIFDYSEIP...KAS'), id='A0A096NNL8', name='<unknown name>', description='PNP', dbxrefs=[]),
 SeqRecord(seq=Seq('MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLADKLTQAQIFDYSEIP...KAS'), id='A0A0D9RVS9', name='<unknown name>', description='PNP', dbxrefs=[]),
 SeqRecord(seq=Seq('MESGVTYEDYEETANWLLSHTSHRPQVAVICGSGLGGLINKLTQAQVFDYSDIP...SAS'), id='A0A1S3ANF8', name='<unknown name>', description='PNP', dbxrefs=[]),
 SeqRecord(seq=Seq('MENRFTYEDYENTAEWLLTHTKHRPHVAVICGSGLGGLTDRLTEAQIFDYSEIP...KTS'), id='A0A250YET9', name='<unknown name>', description='PNP', dbxrefs=[]),
 SeqRecord(seq=Seq('MEDGFMYEDYENTAQWLLSHTKHRPEVAVICGSGLGGLADQLTQAQAFDYSDIP...DNA'), id='A0A286Y5A4', name='<unknown name>', description='PNP', dbxrefs=[]),
 SeqRecord(seq=Seq('MQRGLARSLVSRARRQSHAAASAGRDARGNRAGVAPSEQEGDRYTYEDYKNTAE...KAS'), id='A0A2J8TT03', name='<unknown name>', description='PNP', dbxrefs=[]),
 SeqRecord(seq=Seq('MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLTDKLTQ

In [9]:
all_OS = aaa_OS+bbb_OS+ccc_OS+ddd_OS+eee_OS+fff_OS+ggg_OS+iii_OS+jjj_OS+kkk_OS+lll_OS  #物种信息
all_OS

[SeqRecord(seq=' ', id='A0A096NNL8', name='Papio anubis', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=' ', id='A0A0D9RVS9', name='Chlorocebus sabaeus', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=' ', id='A0A1S3ANF8', name='Erinaceus europaeus', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=' ', id='A0A250YET9', name='Castor canadensis', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=' ', id='A0A286Y5A4', name='Cavia porcellus', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=' ', id='A0A2J8TT03', name='Pongo abelii', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=' ', id='A0A2K5D2N7', name='Aotus nancymaae', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=' ', id='A0A2K5LZC4', name='Cercocebus atys', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=' ', id='A0A2K5QZQ9', name='Cebus imitator', description='<unknown description>', dbxrefs=[])

### 4预处理uniport的序列

In [10]:
class preprocess_uniprot_fasta2pandas(object):
    
    def __init__(self,fasta_list=[],OS_list=[]):
        """初始化"""
        self.fasta_list = fasta_list
        self.OS_list = OS_list
#         self.gene_list = gene_list
#         self.gene_list = ""
        
    
    def fasta_list2fasta_pd(self):
        """在多个关键词的情况下，会有重复的结果，这个方法是为了去重"""
        
#         fasta_dictionary = SeqIO.to_dict(self.fasta_list)
#         protein_sequence =  list(SeqIO.parse(input_fasta_file,"fasta"))


        #------------序列和ID----------------
        pre_pandas_series = {}
        for i in range(len(self.fasta_list)):
            #字典的特性去重
            pre_pandas_series.setdefault(self.fasta_list[i].id,self.fasta_list[i].seq)

        for i,j in  pre_pandas_series.items():
            #将氨基酸连接起来
            pre_pandas_series[i] = "".join(j)

        ID_PSeq2pd = pd.DataFrame(list(pre_pandas_series.items()))
        ID_PSeq2pd.columns = ["Uniprot ID","Protein Sequence"]
#         ID_PSeq2pd.columns = ["Uniprot检索ID","蛋白质序列"]      





        #----------------ID和name-----------------------
        pre2_pandas_series = {}
        for i in range(len(self.OS_list)):
            #字典的特性去重
            pre2_pandas_series.setdefault(self.OS_list[i].id,self.OS_list[i].name)

        ID_Name2pd = pd.DataFrame(list(pre2_pandas_series.items()))
        ID_Name2pd.columns = ["Uniprot ID","Organism"]    
        
        
        #----------------ID和gene----------------------=
        pre3_pandas_series = {}
        for i in range(len(self.fasta_list)):
            #字典的特性去重
            pre3_pandas_series.setdefault(self.fasta_list[i].id,self.fasta_list[i].description)

        ID_GN2pd = pd.DataFrame(list(pre3_pandas_series.items()))
        ID_GN2pd.columns = ["Uniprot ID","Gene Name"]
#
        
        return ID_PSeq2pd,ID_Name2pd,ID_GN2pd

### 定义 文件名字、作用底物名、以及SMILES化学结构式

In [11]:
DLKcat_result_data_output_file_name = "DLKcat_result_data_output_file_name.csv"
Substrate_Name = "Purine_nucleoside"                       
Substrate_SMILES = "C1=C2C(=NC=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)CO)O)O"

In [12]:
pd_fasta_seq,pd_name_id,pd_id_gn = preprocess_uniprot_fasta2pandas(all_fasta_result_list,all_OS).fasta_list2fasta_pd()
pd_fasta_seq

,Uniprot ID,Protein Sequence
0,A0A096NNL8,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLTDKLTEAQ...
1,A0A0D9RVS9,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLADKLTQAQ...
2,A0A1S3ANF8,MESGVTYEDYEETANWLLSHTSHRPQVAVICGSGLGGLINKLTQAQ...
3,A0A250YET9,MENRFTYEDYENTAEWLLTHTKHRPHVAVICGSGLGGLTDRLTEAQ...
4,A0A286Y5A4,MEDGFMYEDYENTAQWLLSHTKHRPEVAVICGSGLGGLADQLTQAQ...
...,...,...
1210,A0A8K0ALK7,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1211,A0A8K0BE76,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1212,A0A8M1FE22,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...
1213,A0A8M1L7W7,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...


In [13]:
pd_name_id

,Uniprot ID,Organism
0,A0A096NNL8,Papio anubis
1,A0A0D9RVS9,Chlorocebus sabaeus
2,A0A1S3ANF8,Erinaceus europaeus
3,A0A250YET9,Castor canadensis
4,A0A286Y5A4,Cavia porcellus
...,...,...
1210,A0A8K0ALK7,Eudyptes robustus
1211,A0A8K0BE76,Eudyptes filholi
1212,A0A8M1FE22,Ursus maritimus
1213,A0A8M1L7W7,Ursus arctos horribilis


In [14]:
pd_id_gn

,Uniprot ID,Gene Name
0,A0A096NNL8,PNP
1,A0A0D9RVS9,PNP
2,A0A1S3ANF8,PNP
3,A0A250YET9,PNP
4,A0A286Y5A4,PNP
...,...,...
1210,A0A8K0ALK7,LACC1
1211,A0A8K0BE76,LACC1
1212,A0A8M1FE22,LACC1
1213,A0A8M1L7W7,LACC1


In [15]:
pd_fasta_seq.insert(loc=1,column="Organism",value=pd_name_id["Organism"])
pd_fasta_seq

,Uniprot ID,Organism,Protein Sequence
0,A0A096NNL8,Papio anubis,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLTDKLTEAQ...
1,A0A0D9RVS9,Chlorocebus sabaeus,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLADKLTQAQ...
2,A0A1S3ANF8,Erinaceus europaeus,MESGVTYEDYEETANWLLSHTSHRPQVAVICGSGLGGLINKLTQAQ...
3,A0A250YET9,Castor canadensis,MENRFTYEDYENTAEWLLTHTKHRPHVAVICGSGLGGLTDRLTEAQ...
4,A0A286Y5A4,Cavia porcellus,MEDGFMYEDYENTAQWLLSHTKHRPEVAVICGSGLGGLADQLTQAQ...
...,...,...,...
1210,A0A8K0ALK7,Eudyptes robustus,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1211,A0A8K0BE76,Eudyptes filholi,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1212,A0A8M1FE22,Ursus maritimus,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...
1213,A0A8M1L7W7,Ursus arctos horribilis,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...


### 5进行DLKcat预测

#### 深度学习权重参数文件载入

In [16]:
fingerprint_dict = model.load_pickle('../pickle/fingerprint_dict.pickle')
atom_dict = model.load_pickle('../pickle/atom_dict.pickle')
bond_dict = model.load_pickle('../pickle/bond_dict.pickle')
edge_dict = model.load_pickle('../pickle/edge_dict.pickle')
word_dict = model.load_pickle('../pickle/sequence_dict.pickle')

In [17]:
DLKcat_raw_input_data = pd_fasta_seq

In [18]:
DLKcat_raw_input_data

,Uniprot ID,Organism,Protein Sequence
0,A0A096NNL8,Papio anubis,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLTDKLTEAQ...
1,A0A0D9RVS9,Chlorocebus sabaeus,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLADKLTQAQ...
2,A0A1S3ANF8,Erinaceus europaeus,MESGVTYEDYEETANWLLSHTSHRPQVAVICGSGLGGLINKLTQAQ...
3,A0A250YET9,Castor canadensis,MENRFTYEDYENTAEWLLTHTKHRPHVAVICGSGLGGLTDRLTEAQ...
4,A0A286Y5A4,Cavia porcellus,MEDGFMYEDYENTAQWLLSHTKHRPEVAVICGSGLGGLADQLTQAQ...
...,...,...,...
1210,A0A8K0ALK7,Eudyptes robustus,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1211,A0A8K0BE76,Eudyptes filholi,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1212,A0A8M1FE22,Ursus maritimus,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...
1213,A0A8M1L7W7,Ursus arctos horribilis,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...


In [19]:
DLKcat_raw_input_data.drop("Uniprot ID",axis=1)

,Organism,Protein Sequence
0,Papio anubis,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLTDKLTEAQ...
1,Chlorocebus sabaeus,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLADKLTQAQ...
2,Erinaceus europaeus,MESGVTYEDYEETANWLLSHTSHRPQVAVICGSGLGGLINKLTQAQ...
3,Castor canadensis,MENRFTYEDYENTAEWLLTHTKHRPHVAVICGSGLGGLTDRLTEAQ...
4,Cavia porcellus,MEDGFMYEDYENTAQWLLSHTKHRPEVAVICGSGLGGLADQLTQAQ...
...,...,...
1210,Eudyptes robustus,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1211,Eudyptes filholi,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1212,Ursus maritimus,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...
1213,Ursus arctos horribilis,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...


In [20]:
DLKcat_raw_input_data.drop("Organism",axis=1)

,Uniprot ID,Protein Sequence
0,A0A096NNL8,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLTDKLTEAQ...
1,A0A0D9RVS9,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLADKLTQAQ...
2,A0A1S3ANF8,MESGVTYEDYEETANWLLSHTSHRPQVAVICGSGLGGLINKLTQAQ...
3,A0A250YET9,MENRFTYEDYENTAEWLLTHTKHRPHVAVICGSGLGGLTDRLTEAQ...
4,A0A286Y5A4,MEDGFMYEDYENTAQWLLSHTKHRPEVAVICGSGLGGLADQLTQAQ...
...,...,...
1210,A0A8K0ALK7,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1211,A0A8K0BE76,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1212,A0A8M1FE22,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...
1213,A0A8M1L7W7,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...


In [21]:
class DLKcat_input_preprocess(object):
    """为了方便预测需要将前面爬取到的数据进行适当格式处理"""
    def __init__(self,DLKcat_raw_input_data):
#         pass
        self.DLKcat_raw_input_data = DLKcat_raw_input_data
    
    def pandas_preprocess(self):
        #将uniprot上符合条件的fasta的Dataframe转换
        
        global Substrate_Name,Substrate_SMILES
        
        self.DLKcat_raw_input_data = DLKcat_raw_input_data.drop("Uniprot ID",axis=1)
        self.DLKcat_raw_input_data = self.DLKcat_raw_input_data.drop("Organism",axis=1)
#         print(self.DLKcat_raw_input_data)
        self.DLKcat_raw_input_data.insert(loc=0,column="Substrate Name",value=Substrate_Name)
        self.DLKcat_raw_input_data.insert(loc=1,column="Substrate SMILES",value=Substrate_SMILES)
#         self.DLKcat_raw_input_data.columns = ["Uniprot ID","Substrate Name","Substrate SMILES","Protein Sequence"]


        self.DLKcat_raw_input_data.to_csv("input_Kcat_file.csv", header=True, sep=',', index=False)
#         print(self.DLKcat_raw_input_data)
        
        return print("DLKcat预测函数的数据预处理完成")

In [22]:
DLKcat_input_preprocess(DLKcat_raw_input_data).pandas_preprocess()

DLKcat预测函数的数据预处理完成


In [23]:
DLKcat_raw_input_data

,Uniprot ID,Organism,Protein Sequence
0,A0A096NNL8,Papio anubis,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLTDKLTEAQ...
1,A0A0D9RVS9,Chlorocebus sabaeus,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLADKLTQAQ...
2,A0A1S3ANF8,Erinaceus europaeus,MESGVTYEDYEETANWLLSHTSHRPQVAVICGSGLGGLINKLTQAQ...
3,A0A250YET9,Castor canadensis,MENRFTYEDYENTAEWLLTHTKHRPHVAVICGSGLGGLTDRLTEAQ...
4,A0A286Y5A4,Cavia porcellus,MEDGFMYEDYENTAQWLLSHTKHRPEVAVICGSGLGGLADQLTQAQ...
...,...,...,...
1210,A0A8K0ALK7,Eudyptes robustus,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1211,A0A8K0BE76,Eudyptes filholi,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1212,A0A8M1FE22,Ursus maritimus,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...
1213,A0A8M1L7W7,Ursus arctos horribilis,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...


In [24]:
pd_fasta_seq

,Uniprot ID,Organism,Protein Sequence
0,A0A096NNL8,Papio anubis,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLTDKLTEAQ...
1,A0A0D9RVS9,Chlorocebus sabaeus,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLADKLTQAQ...
2,A0A1S3ANF8,Erinaceus europaeus,MESGVTYEDYEETANWLLSHTSHRPQVAVICGSGLGGLINKLTQAQ...
3,A0A250YET9,Castor canadensis,MENRFTYEDYENTAEWLLTHTKHRPHVAVICGSGLGGLTDRLTEAQ...
4,A0A286Y5A4,Cavia porcellus,MEDGFMYEDYENTAQWLLSHTKHRPEVAVICGSGLGGLADQLTQAQ...
...,...,...,...
1210,A0A8K0ALK7,Eudyptes robustus,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1211,A0A8K0BE76,Eudyptes filholi,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1212,A0A8M1FE22,Ursus maritimus,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...
1213,A0A8M1L7W7,Ursus arctos horribilis,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...


In [25]:
DLKcat_raw_input_data.insert(loc=1,column="Substrate Name",value=Substrate_Name)
DLKcat_raw_input_data.insert(loc=2,column="Substrate_SMILES",value=Substrate_SMILES)
DLKcat_raw_input_data

,Uniprot ID,Substrate Name,Substrate_SMILES,Organism,Protein Sequence
0,A0A096NNL8,Purine_nucleoside,C1=C2C(=NC=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H]...,Papio anubis,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLTDKLTEAQ...
1,A0A0D9RVS9,Purine_nucleoside,C1=C2C(=NC=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H]...,Chlorocebus sabaeus,MENGYTYEDYKNTAEWLLSHTKHRPQVAIICGSGLGGLADKLTQAQ...
2,A0A1S3ANF8,Purine_nucleoside,C1=C2C(=NC=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H]...,Erinaceus europaeus,MESGVTYEDYEETANWLLSHTSHRPQVAVICGSGLGGLINKLTQAQ...
3,A0A250YET9,Purine_nucleoside,C1=C2C(=NC=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H]...,Castor canadensis,MENRFTYEDYENTAEWLLTHTKHRPHVAVICGSGLGGLTDRLTEAQ...
4,A0A286Y5A4,Purine_nucleoside,C1=C2C(=NC=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H]...,Cavia porcellus,MEDGFMYEDYENTAQWLLSHTKHRPEVAVICGSGLGGLADQLTQAQ...
...,...,...,...,...,...
1210,A0A8K0ALK7,Purine_nucleoside,C1=C2C(=NC=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H]...,Eudyptes robustus,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1211,A0A8K0BE76,Purine_nucleoside,C1=C2C(=NC=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H]...,Eudyptes filholi,MVEAVLLDLFSLPANLQSNIQGLLRNTLETIEKCSSIPAPFVYVMC...
1212,A0A8M1FE22,Purine_nucleoside,C1=C2C(=NC=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H]...,Ursus maritimus,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...
1213,A0A8M1L7W7,Purine_nucleoside,C1=C2C(=NC=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H]...,Ursus arctos horribilis,MAEAVLIDFFGLKFNSQKNSHQALLKTLNAVRYHHAAKAKFLCIMC...


#### Kcat催化常数预测主函数

In [26]:
def split_sequence(sequence, ngram):
    sequence = '-' + sequence + '='
    # print(sequence)
    # words = [word_dict[sequence[i:i+ngram]] for i in range(len(sequence)-ngram+1)]

    words = list()
    for i in range(len(sequence)-ngram+1) :
        try :
            words.append(word_dict[sequence[i:i+ngram]])
        except :
            word_dict[sequence[i:i+ngram]] = 0
            words.append(word_dict[sequence[i:i+ngram]])

    return np.array(words)
    # return word_dict


def create_atoms(mol):
    """Create a list of atom (e.g., hydrogen and oxygen) IDs
    considering the aromaticity."""
    # atom_dict = defaultdict(lambda: len(atom_dict))
    atoms = [a.GetSymbol() for a in mol.GetAtoms()]
    # print(atoms)
    for a in mol.GetAromaticAtoms():
        i = a.GetIdx()
        atoms[i] = (atoms[i], 'aromatic')
    atoms = [atom_dict[a] for a in atoms]
    # atoms = list()
    # for a in atoms :
    #     try: 
    #         atoms.append(atom_dict[a])
    #     except :
    #         atom_dict[a] = 0
    #         atoms.append(atom_dict[a])

    return np.array(atoms)


def create_ijbonddict(mol):
    """Create a dictionary, which each key is a node ID
    and each value is the tuples of its neighboring node
    and bond (e.g., single and double) IDs."""
    # bond_dict = defaultdict(lambda: len(bond_dict))
    i_jbond_dict = defaultdict(lambda: [])
    for b in mol.GetBonds():
        i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
        bond = bond_dict[str(b.GetBondType())]
        i_jbond_dict[i].append((j, bond))
        i_jbond_dict[j].append((i, bond))
    return i_jbond_dict


def extract_fingerprints(atoms, i_jbond_dict, radius):
    """Extract the r-radius subgraphs (i.e., fingerprints)
    from a molecular graph using Weisfeiler-Lehman algorithm."""

    # fingerprint_dict = defaultdict(lambda: len(fingerprint_dict))
    # edge_dict = defaultdict(lambda: len(edge_dict))

    if (len(atoms) == 1) or (radius == 0):
        fingerprints = [fingerprint_dict[a] for a in atoms]

    else:
        nodes = atoms
        i_jedge_dict = i_jbond_dict

        for _ in range(radius):

            """Update each node ID considering its neighboring nodes and edges
            (i.e., r-radius subgraphs or fingerprints)."""
            fingerprints = []
            for i, j_edge in i_jedge_dict.items():
                neighbors = [(nodes[j], edge) for j, edge in j_edge]
                fingerprint = (nodes[i], tuple(sorted(neighbors)))
                # fingerprints.append(fingerprint_dict[fingerprint])
                # fingerprints.append(fingerprint_dict.get(fingerprint))
                try :
                    fingerprints.append(fingerprint_dict[fingerprint])
                except :
                    fingerprint_dict[fingerprint] = 0
                    fingerprints.append(fingerprint_dict[fingerprint])

            nodes = fingerprints

            """Also update each edge ID considering two nodes
            on its both sides."""
            _i_jedge_dict = defaultdict(lambda: [])
            for i, j_edge in i_jedge_dict.items():
                for j, edge in j_edge:
                    both_side = tuple(sorted((nodes[i], nodes[j])))
                    # edge = edge_dict[(both_side, edge)]
                    # edge = edge_dict.get((both_side, edge))
                    try :
                        edge = edge_dict[(both_side, edge)]
                    except :
                        edge_dict[(both_side, edge)] = 0
                        edge = edge_dict[(both_side, edge)]

                    _i_jedge_dict[i].append((j, edge))
            i_jedge_dict = _i_jedge_dict

    return np.array(fingerprints)


def create_adjacency(mol):
    adjacency = Chem.GetAdjacencyMatrix(mol)
    return np.array(adjacency)

def dump_dictionary(dictionary, filename):
    with open(filename, 'wb') as file:
        pickle.dump(dict(dictionary), file)

def load_tensor(file_name, dtype):
    return [dtype(d).to(device) for d in np.load(file_name + '.npy', allow_pickle=True)]


class Predictor(object):
    def __init__(self, model):
        self.model = model

    def predict(self, data):
        predicted_value = self.model.forward(data)

        return predicted_value


# One method to obtain SMILES by PubChem API using the website
def get_smiles(name):
    # smiles = redis_cli.get(name)
    # if smiles is None:
    try :
        url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/%s/property/CanonicalSMILES/TXT' % name
        req = requests.get(url)
        if req.status_code != 200:
            smiles = None
        else:
            smiles = req.content.splitlines()[0].decode()
            # print(smiles)
        # redis_cli.set(name, smiles, ex=None)

        # print smiles
    except :
        smiles = None

    # name_smiles[name] = smiles
    return smiles

def main() :
#     name = sys.argv[1:][0]
    #改动3
#     name = input_Kcat_file
#     print(name)
    # with open('./input.tsv', 'r') as infile :
#     with open(name, 'r') as infile :
    with open("./input_Kcat_file.csv", 'r') as infile :    
        lines = infile.readlines()
#         print(lines)

    fingerprint_dict = model.load_pickle('../pickle/fingerprint_dict.pickle')
    atom_dict = model.load_pickle('../pickle/atom_dict.pickle')
    bond_dict = model.load_pickle('../pickle/bond_dict.pickle')
    word_dict = model.load_pickle('../pickle/sequence_dict.pickle')
    n_fingerprint = len(fingerprint_dict)
    n_word = len(word_dict)

    radius=2
    ngram=3

    dim=10
    layer_gnn=3
    side=5
    window=11
    layer_cnn=3
    layer_output=3
    lr=1e-3
    lr_decay=0.5
    decay_interval=10
    weight_decay=1e-6
    iteration=100

    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    # torch.manual_seed(1234)
    Kcat_model = model.KcatPrediction(device, n_fingerprint, n_word, 2*dim, layer_gnn, window, layer_cnn, layer_output).to(device)
    #改动2
    Kcat_model.load_state_dict(torch.load('../pickle/all--radius2--ngram3--dim20--layer_gnn3--window11--layer_cnn3--layer_output3--lr1e-3--lr_decay0.5--decay_interval10--weight_decay1e-6--iteration50', map_location=device))
    # print(state_dict.keys())
    # model.eval()
    predictor = Predictor(Kcat_model)

    print('It\'s time to start the prediction!')
    print("""
          """)
    print('-----------------------------------')

    i = 0
  
    with open(DLKcat_result_data_output_file_name, 'w') as outfile :
#     with open('./output.tsv', 'w') as outfile :
        items = ['Substrate Name', 'Substrate SMILES', 'Protein Sequence', 'Kcat value (1/s)']
        outfile.write(','.join(items)+'\n')
#         print("test1")
        for line in lines[1:] :
            line_item = list()
            data = line.strip().split(',')
#             print("test1")
            # i += 1
            # print('This is', i, '---------------------------------------')
#             print(data)
#             print(data[0])
#             print(data[1])
#             print(data[2])
#             break

            name = data[0]
            smiles = data[1]
            sequence = data[2]
#             print("test2")
            if smiles and smiles != 'None' :
                smiles = data[1]
            else :
                smiles = get_smiles(name)
            # print(smiles)

            try :
                if "." not in smiles :
#                     i += 1
#                     print('This is',i)

                    mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
                    atoms = create_atoms(mol)
                    # print(atoms)
                    i_jbond_dict = create_ijbonddict(mol)
                    # print(i_jbond_dict)

                    fingerprints = extract_fingerprints(atoms, i_jbond_dict, radius)
                    # print(fingerprints)
                    # compounds.append(fingerprints)

                    adjacency = create_adjacency(mol)
                    # print(adjacency)
                    # adjacencies.append(adjacency)

                    words = split_sequence(sequence,ngram)
                    # print(words)
                    # proteins.append(words)

                    fingerprints = torch.LongTensor(fingerprints)
                    adjacency = torch.FloatTensor(adjacency)
                    words = torch.LongTensor(words)

                    inputs = [fingerprints, adjacency, words]

                    prediction = predictor.predict(inputs)
                    Kcat_log_value = prediction.item()
                    Kcat_value = '%.4f' %math.pow(2,Kcat_log_value)
#                     print(Kcat_value)
                    line_item = [name,smiles,sequence,Kcat_value]

                    outfile.write(','.join(line_item)+'\n')
            except :
                Kcat_value = 'None'
#                 print(Kcat_value)
                line_item = [name,smiles,sequence,Kcat_value]
                outfile.write(','.join(line_item)+'\n')
    print('Prediction success!')      

#### 进行Kcat预测

In [ ]:
main()

It's time to start the prediction!

          
-----------------------------------


## 6对DLKcat的结果文件加上uniprot ID 和物种信息

In [ ]:
DLKcat_result_data = pd.read_csv("./DLKcat_result_data_output_file_name.csv")

In [ ]:
DLKcat_result_data

In [ ]:
DLKcat_raw_input_data

In [ ]:
DLKcat_result_data.insert(loc=0,column="Uniprot ID",value=DLKcat_raw_input_data["Uniprot ID"])
DLKcat_result_data.insert(loc=1,column="Organism",value=DLKcat_raw_input_data["Organism"])
DLKcat_result_data.insert(loc=2,column="Gene Name",value=pd_id_gn["Gene Name"])

In [ ]:
DLKcat_result_data

### 7输出最终结果文件

In [ ]:
DLKcat_result_data

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#输出格式为csv
# DLKcat_result_data.to_csv("C:/Users/1/Desktop/WeiLab/code/cal_result/1023-1531.csv", header=True, sep=',', index=False)

#输出格式为xls
DLKcat_result_data.to_excel("C:/Users/1/Desktop/WeiLab/code/cal_result/test_demo.xls",index=False,header=True)